In [ ]:
import fitz
from fitz import Page
import numpy as np
import pandas as pd
from io import StringIO

In [ ]:
def add_nulls(search: list):
    discount_idxs = []
    array = search.copy()
    for i, word in enumerate(array):
        if word == "Rabat":
            discount_idxs.append(i)
    for index in reversed(discount_idxs):
        array[index + 1:index + 1] = ["0", "0"]
    return array

In [ ]:
def crop_pdf(passed_page: Page, start_word: str, end_word: str):
    page_width = round(passed_page.rect.width)
    start_rect = passed_page.get_textpage_ocr(3, "pol", 300, True).search(start_word)[0].rect
    end_rect = passed_page.get_textpage_ocr(3, "pol", 300, True).search(end_word)[0].rect
    start_y = start_rect.round().bottom_left[1]
    end_y = end_rect.round().top_left[1]
    passed_page.set_cropbox(fitz.Rect(0, start_y, page_width, end_y))


In [ ]:
def sum_discount(arg_pd: pd.DataFrame):
    dfp = arg_pd.copy()
    x = dfp[dfp["Nazwa"] == "Rabat"]
    for i in x.index.array:
        main_price = dfp.loc[i - 1, "Wartość"]
        discount_price = x["Cena"][i]
        new_price = np.round(main_price + discount_price, decimals=2)
        dfp.loc[i - 1, "Wartość"] = new_price
    dfp = dfp.drop(x.index.array)
    return dfp


In [ ]:
def generate_df(file_name: str):
    doc = fitz.open(f"input/{file_name}.pdf")
    page = list(doc.pages())[0]
    crop_pdf(page, "NIEFISKALNY", "Sprzed")
    text_page = page.get_textpage_ocr(3, "pol", 300, True)
    text = text_page.extractText()
    lines = text.split('\n')
    lines = add_nulls(lines)
    grouped_lines = [';'.join(lines[i:i+5]) for i in range(0, len(lines), 5)]
    grouped_lines = add_nulls(grouped_lines)
    result = '\n'.join(grouped_lines)
    result = result.encode("utf-8").decode()
    result = StringIO(result)
    df_new = pd.read_table(result, sep=";")
    df_new["Ilość"] = df_new["Ilość"].map(lambda row: row.split(" x")[0])
    ints = df_new[["Ilość", "Cena", "Wartość"]]
    ints = ints.map(lambda series: series.replace(",", ".")).map(np.float32)
    df_new.update(ints)
    df_new = sum_discount(df_new)
    df_new = df_new.reset_index(drop=True)
    doc.close()
    return df_new

In [ ]:
df = generate_df("a")
print(df)